In [2]:
import requests
import pandas as pd
import json
from datetime import datetime
import time

def download_dsire_policies():
    """Download complete US renewable energy policy dataset from DSIRE API"""
    
    base_url = "http://programs.dsireusa.org/api/v1/getprograms"
    formats = ['json', 'xml']
    
    # Download JSON version (easier to parse)
    json_url = f"{base_url}/json"
    
    print(f"Downloading DSIRE data from {json_url}...")
    
    try:
        response = requests.get(json_url, timeout=60)
        response.raise_for_status()
        
        # Save raw JSON
        with open('dsire_raw.json', 'w', encoding='utf-8') as f:
            json.dump(response.json(), f, indent=2)
        
        # Parse to DataFrame
        data = response.json()
        
        # Extract programs (structure may vary; adjust based on actual response)
        if isinstance(data, list):
            programs = data
        elif isinstance(data, dict) and 'programs' in data:
            programs = data['programs']
        else:
            programs = [data]
        
        df = pd.DataFrame(programs)
        
        # Clean and standardize columns
        rename_map = {
            'ProgramID': 'program_id',
            'ProgramName': 'program_name',
            'State': 'state',
            'PolicyType': 'policy_type',
            'Sector': 'sector',
            'StartDate': 'start_date',
            'LastUpdate': 'last_updated',
            'ImplementingSector': 'implementing_sector',
            'Category': 'category',
            'Type': 'type',
            'ProgramDescription': 'description'
        }
        
        df.rename(columns=rename_map, inplace=True)
        
        # Parse dates
        date_cols = ['start_date', 'last_updated']
        for col in date_cols:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors='coerce')
        
        # Add metadata
        df['download_date'] = datetime.now()
        df['country'] = 'US'
        
        # Save to CSV
        df.to_csv('dsire_policies.csv', index=False)
        
        print(f"✓ Downloaded {len(df)} DSIRE policy records")
        print(f"  Date range: {df['start_date'].min()} to {df['start_date'].max()}")
        print(f"  Saved to: dsire_policies.csv")
        
        return df
        
    except Exception as e:
        print(f"✗ Error downloading DSIRE data: {e}")
        return None


def download_dsire_by_daterange(start_date, end_date):
    """Download DSIRE policies for specific date range"""
    
    # Format: YYYYMMDD
    start_str = start_date.strftime('%Y%m%d')
    end_str = end_date.strftime('%Y%m%d')
    
    url = f"http://programs.dsireusa.org/api/v1/getprogramsbydate/{start_str}/{end_str}/json"
    
    print(f"Downloading DSIRE data from {start_date} to {end_date}...")
    
    try:
        response = requests.get(url, timeout=60)
        response.raise_for_status()
        
        data = response.json()
        df = pd.DataFrame(data if isinstance(data, list) else [data])
        
        print(f"✓ Downloaded {len(df)} records for date range")
        return df
        
    except Exception as e:
        print(f"✗ Error: {e}")
        return None


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import re
import time

def scrape_nsdl_fortnightly_reports():
    """Scrape all NSDL fortnightly sector-wise FPI reports from 2012 to present"""
    
    base_url = "https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/"
    
    # Generate all fortnightly dates from June 2012 to present
    start_date = datetime(2012, 6, 1)
    end_date = datetime.now()
    
    all_data = []
    
    # Typical file naming pattern: FIIInvestSector_MMMDDYYYY.html
    # e.g., FIIInvestSector_JUL152023.html
    
    current = start_date
    attempts = 0
    successful = 0
    
    while current <= end_date:
        # Try both 1st and 15th of each month (fortnightly)
        for day in [1, 15]:
            try:
                date_obj = datetime(current.year, current.month, day)
                
                # Format date for filename: JUL152023
                month_abbr = date_obj.strftime('%b').upper()
                day_str = f"{day:02d}"
                year_str = str(date_obj.year)
                
                filename = f"FIIInvestSector_{month_abbr}{day_str}{year_str}.html"
                url = base_url + filename
                
                print(f"Attempting: {url}")
                
                response = requests.get(url, timeout=30)
                
                if response.status_code == 200:
                    soup = BeautifulSoup(response.content, 'html.parser')
                    
                    # Find the main data table
                    tables = soup.find_all('table')
                    
                    for table in tables:
                        # Parse table to DataFrame
                        df_temp = pd.read_html(str(table))[0]
                        
                        # Check if this is the sector data table
                        if 'Sector' in str(df_temp.columns) or 'SECTOR' in str(df_temp.columns):
                            df_temp['report_date'] = date_obj
                            df_temp['report_url'] = url
                            all_data.append(df_temp)
                            successful += 1
                            print(f"  ✓ Downloaded {date_obj.strftime('%Y-%m-%d')}")
                            break
                    
                else:
                    print(f"  ✗ Not found (status {response.status_code})")
                
                attempts += 1
                time.sleep(1)  # Respectful scraping delay
                
            except Exception as e:
                print(f"  ✗ Error: {e}")
        
        # Move to next month
        if current.month == 12:
            current = datetime(current.year + 1, 1, 1)
        else:
            current = datetime(current.year, current.month + 1, 1)
    
    # Combine all data
    if all_data:
        df_combined = pd.concat(all_data, ignore_index=True)
        
        # Standardize column names
        df_combined.columns = df_combined.columns.str.lower().str.strip().str.replace(' ', '_')
        
        # Save to CSV
        df_combined.to_csv('nsdl_fpi_fortnightly.csv', index=False)
        
        print(f"\n✓ Successfully scraped {successful} reports out of {attempts} attempts")
        print(f"  Total rows: {len(df_combined)}")
        print(f"  Saved to: nsdl_fpi_fortnightly.csv")
        
        return df_combined
    else:
        print("✗ No data collected")
        return None


def scrape_nsdl_interactive_page():
    """Alternative: scrape from the interactive selection page"""
    
    url = "https://www.fpi.nsdl.co.in/web/Reports/FPI_Fortnightly_Selection.aspx"
    
    print(f"Accessing NSDL interactive page: {url}")
    
    try:
        response = requests.get(url, timeout=30)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all report links
        links = soup.find_all('a', href=True)
        
        report_links = []
        for link in links:
            href = link['href']
            if 'FIIInvest' in href or 'Sector' in href:
                full_url = f"https://www.fpi.nsdl.co.in{href}" if href.startswith('/') else href
                report_links.append(full_url)
        
        print(f"Found {len(report_links)} report links")
        
        # Download each report
        all_data = []
        for i, link in enumerate(report_links[:10], 1):  # Limit to first 10 for testing
            print(f"Downloading {i}/{len(report_links)}: {link}")
            
            try:
                resp = requests.get(link, timeout=30)
                soup = BeautifulSoup(resp.content, 'html.parser')
                
                tables = pd.read_html(resp.content)
                if tables:
                    df = tables[0]
                    df['source_url'] = link
                    all_data.append(df)
                    print(f"  ✓ Got {len(df)} rows")
                
                time.sleep(1)
                
            except Exception as e:
                print(f"  ✗ Error: {e}")
        
        if all_data:
            df_combined = pd.concat(all_data, ignore_index=True)
            df_combined.to_csv('nsdl_fpi_interactive.csv', index=False)
            return df_combined
        
    except Exception as e:
        print(f"✗ Error accessing interactive page: {e}")
    
    return None


In [1]:
def download_mnre_notices():
    """Download high-priority MNRE policy notices"""
    
    # Define high-priority notices from MNRE site
    notices = [
        {
            'date': '2025-10-13',
            'title': 'ALMM Order Solar PV Modules Update',
            'url': 'https://mnre.gov.in/img/documents/uploads/file_f-1728822686315.pdf',
            'policy_type': 'regulatory',
            'tech_focus': 'solar_pv',
            'importance': 'high'
        },
        {
            'date': '2025-09-23',
            'title': 'ALMM List-II for Solar PV Cells',
            'url': 'https://mnre.gov.in/img/documents/uploads/file_f-1727085527749.pdf',
            'policy_type': 'regulatory',
            'tech_focus': 'solar_pv',
            'importance': 'high'
        },
        {
            'date': '2025-10-24',
            'title': 'RE Equipment Import Monitoring System',
            'url': 'https://mnre.gov.in/img/documents/uploads/file_f-1729764869508.pdf',
            'policy_type': 'regulatory',
            'tech_focus': 'renewable_general',
            'importance': 'high'
        },
        {
            'date': '2025-10-10',
            'title': 'Solar Systems Import Policy Amendment',
            'url': 'https://mnre.gov.in/img/documents/uploads/file_f-1728562639089.pdf',
            'policy_type': 'regulatory',
            'tech_focus': 'solar',
            'importance': 'high'
        },
        {
            'date': '2025-09-17',
            'title': 'Solar Park Scheme Extension',
            'url': 'https://mnre.gov.in/img/documents/uploads/file_f-1726567437453.pdf',
            'policy_type': 'operational',
            'tech_focus': 'solar',
            'importance': 'medium'
        },
        {
            'date': '2025-09-16',
            'title': 'National Bioenergy Programme Budget Increase',
            'url': 'https://mnre.gov.in/img/documents/uploads/file_f-1726475834060.pdf',
            'policy_type': 'fiscal',
            'tech_focus': 'bioenergy',
            'importance': 'medium'
        }
    ]
    
    df_notices = pd.DataFrame(notices)
    df_notices['announcement_date'] = pd.to_datetime(df_notices['date'])
    df_notices['country'] = 'India'
    df_notices['source'] = 'MNRE'
    
    # Download PDFs
    import os
    os.makedirs('mnre_pdfs', exist_ok=True)
    
    for idx, row in df_notices.iterrows():
        try:
            print(f"Downloading: {row['title']}")
            response = requests.get(row['url'], timeout=60)
            
            filename = f"mnre_pdfs/{row['date']}_{row['title'][:30].replace(' ', '_')}.pdf"
            with open(filename, 'wb') as f:
                f.write(response.content)
            
            df_notices.loc[idx, 'local_file'] = filename
            print(f"  ✓ Saved to {filename}")
            
            time.sleep(2)
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    # Save metadata
    df_notices.to_csv('mnre_policy_events.csv', index=False)
    print(f"\n✓ Saved {len(df_notices)} MNRE policy events to mnre_policy_events.csv")
    
    return df_notices


In [2]:
def download_eu_red_timeline():
    """Download EU RED timeline PDF and code policy events"""
    
    url = "https://eeb.org/wp-content/uploads/2023/11/RED-Timeline-Guidance-transposition-and-implementation-deadlines-1.pdf"
    
    print(f"Downloading EU RED timeline from {url}")
    
    try:
        response = requests.get(url, timeout=60)
        response.raise_for_status()
        
        with open('eu_red_timeline.pdf', 'wb') as f:
            f.write(response.content)
        
        print("✓ Downloaded eu_red_timeline.pdf")
        
        # Manual coding of key EU policy events
        eu_events = [
            {
                'event_date': '2023-11-20',
                'event_type': 'directive_entry_force',
                'description': 'RED III entry into force',
                'target_change': '42.5% renewable energy by 2030',
                'jurisdiction': 'EU',
                'importance': 'high'
            },
            {
                'event_date': '2021-07-14',
                'event_type': 'policy_proposal',
                'description': 'Fit for 55 package proposal',
                'target_change': '40% renewable energy by 2030 (later increased)',
                'jurisdiction': 'EU',
                'importance': 'high'
            },
            {
                'event_date': '2022-05-18',
                'event_type': 'policy_proposal',
                'description': 'REPowerEU Plan announcement',
                'target_change': 'Accelerated renewable deployment, 45% target',
                'jurisdiction': 'EU',
                'importance': 'high'
            },
            {
                'event_date': '2024-07-01',
                'event_type': 'transposition_deadline',
                'description': 'Member state transposition deadline (Article 36)',
                'target_change': None,
                'jurisdiction': 'EU',
                'importance': 'medium'
            },
            {
                'event_date': '2025-05-21',
                'event_type': 'transposition_deadline',
                'description': 'Transposition deadline for specific provisions',
                'target_change': None,
                'jurisdiction': 'EU',
                'importance': 'medium'
            }
        ]
        
        df_eu = pd.DataFrame(eu_events)
        df_eu['announcement_date'] = pd.to_datetime(df_eu['event_date'])
        df_eu['country'] = 'EU'
        df_eu['source'] = 'EU_RED'
        
        df_eu.to_csv('eu_policy_events.csv', index=False)
        print(f"✓ Coded {len(df_eu)} EU policy events to eu_policy_events.csv")
        
        return df_eu
        
    except Exception as e:
        print(f"✗ Error: {e}")
        return None


In [6]:
def run_full_pipeline():
    """Execute complete data collection pipeline"""
    
    print("="*80)
    print("RENEWABLE ENERGY POLICY & FPI FLOW DATA COLLECTION PIPELINE")
    print("="*80)
    
    # 1. US Policies (DSIRE)
    print("\n[1/4] Downloading US policy data (DSIRE)...")
    df_dsire = download_dsire_policies()
    
    # 2. India FPI Flows (NSDL)
    print("\n[2/4] Scraping India FPI sector flows (NSDL)...")
    df_nsdl = scrape_nsdl_fortnightly_reports()
    
    # 3. India Policies (MNRE)
    print("\n[3/4] Downloading India policy notices (MNRE)...")
    df_mnre = download_mnre_notices()
    
    # 4. EU Policies
    print("\n[4/4] Processing EU RED timeline...")
    df_eu = download_eu_red_timeline()
    
    # Combine policy events
    print("\n" + "="*80)
    print("CONSOLIDATING POLICY EVENTS")
    print("="*80)
    
    policy_dfs = []
    
    if df_dsire is not None:
        df_dsire_clean = df_dsire[['program_id', 'program_name', 'state', 'policy_type', 
                                     'sector', 'start_date', 'country']].copy()
        df_dsire_clean.rename(columns={'program_name': 'description', 
                                        'start_date': 'announcement_date'}, inplace=True)
        df_dsire_clean['source'] = 'DSIRE'
        policy_dfs.append(df_dsire_clean)
    
    if df_mnre is not None:
        policy_dfs.append(df_mnre[['title', 'announcement_date', 'policy_type', 
                                     'tech_focus', 'country', 'source', 'importance']])
    
    if df_eu is not None:
        policy_dfs.append(df_eu)
    
    if policy_dfs:
        df_all_policies = pd.concat(policy_dfs, ignore_index=True, sort=False)
        df_all_policies.to_csv('master_policy_events.csv', index=False)
        
        print(f"✓ Master policy database created: {len(df_all_policies)} events")
        print(f"  Countries: {df_all_policies['country'].value_counts().to_dict()}")
        print(f"  Date range: {df_all_policies['announcement_date'].min()} to {df_all_policies['announcement_date'].max()}")
    
    print("\n" + "="*80)
    print("PIPELINE COMPLETE")
    print("="*80)
    print("\nFiles created:")
    print("  - dsire_policies.csv (US policy data)")
    print("  - nsdl_fpi_fortnightly.csv (India FPI flows)")
    print("  - mnre_policy_events.csv (India policy events)")
    print("  - eu_policy_events.csv (EU policy events)")
    print("  - master_policy_events.csv (consolidated policy database)")
    
    return {
        'dsire': df_dsire,
        'nsdl': df_nsdl,
        'mnre': df_mnre,
        'eu': df_eu
    }


# Execute pipeline
if __name__ == "__main__":
    results = run_full_pipeline()


RENEWABLE ENERGY POLICY & FPI FLOW DATA COLLECTION PIPELINE

[1/4] Downloading US policy data (DSIRE)...
✗ Error downloading DSIRE data: 500 Server Error: Internal Server Error for url: http://programs.dsireusa.org/api/v1/getprograms/json

[2/4] Scraping India FPI sector flows (NSDL)...
Attempting: https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_JUN012012.html
  ✗ Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Attempting: https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_JUN152012.html
  ✗ Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Attempting: https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_JUL012012.html
  ✗ Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Atte

In [7]:
import pandas as pd
import requests

# SEBI provides cleaner, downloadable FPI data
sebi_url = "https://www.sebi.gov.in/statistics/fpi-investment/fortnightly-sector-wise.html"

# Download quarterly summary (easier to work with)
quarterly_url = "https://www.fpi.nsdl.co.in/Reports/Yearwise.aspx?RptType=6"

print("Visit these URLs in your browser and download manually:")
print(f"1. SEBI FPI Statistics: {sebi_url}")
print(f"2. NSDL Quarterly Data: {quarterly_url}")


Visit these URLs in your browser and download manually:
1. SEBI FPI Statistics: https://www.sebi.gov.in/statistics/fpi-investment/fortnightly-sector-wise.html
2. NSDL Quarterly Data: https://www.fpi.nsdl.co.in/Reports/Yearwise.aspx?RptType=6


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_nsdl_slowly():
    """Scrape with long delays and retries"""
    
    base_url = "https://www.fpi.nsdl.co.in/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/"
    
    # Use browser-like headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.9',
        'Connection': 'keep-alive'
    }
    
    session = requests.Session()
    
    # Start with known recent dates (last 6 months only)
    test_dates = [
        ('NOV', 15, 2025),
        ('OCT', 31, 2025),
        ('OCT', 15, 2025),
        ('SEP', 30, 2025),
        ('SEP', 15, 2025),
        ('AUG', 31, 2025),
    ]
    
    all_data = []
    
    for month, day, year in test_dates:
        filename = f"FIIInvestSector_{month}{day:02d}{year}.html"
        url = base_url + filename
        
        print(f"Trying: {url}")
        
        try:
            time.sleep(5)  # 5 second delay between requests
            
            response = session.get(url, headers=headers, timeout=30)
            
            if response.status_code == 200:
                # Parse tables
                tables = pd.read_html(response.content)
                
                if tables:
                    df = tables[0]  # Usually first table is the data
                    df['report_date'] = f"{year}-{month}-{day:02d}"
                    df['source_url'] = url
                    all_data.append(df)
                    
                    print(f"  ✓ Success! Got {len(df)} rows")
                    
                    # Save immediately
                    df.to_csv(f"nsdl_{year}_{month}_{day}.csv", index=False)
            else:
                print(f"  ✗ Status {response.status_code}")
                
        except Exception as e:
            print(f"  ✗ Error: {e}")
            time.sleep(10)  # Longer delay after error
    
    if all_data:
        combined = pd.concat(all_data, ignore_index=True)
        combined.to_csv('nsdl_combined_manual.csv', index=False)
        print(f"\n✓ Total downloaded: {len(all_data)} reports")
        return combined
    
    return None

# Run with delays
nsdl_data = scrape_nsdl_slowly()


Trying: https://www.fpi.nsdl.co.in/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_NOV152025.html
  ✓ Success! Got 29 rows
Trying: https://www.fpi.nsdl.co.in/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_OCT312025.html
  ✓ Success! Got 29 rows
Trying: https://www.fpi.nsdl.co.in/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_OCT152025.html
  ✓ Success! Got 29 rows
Trying: https://www.fpi.nsdl.co.in/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_SEP302025.html
  ✓ Success! Got 29 rows
Trying: https://www.fpi.nsdl.co.in/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_SEP152025.html
  ✗ Error: HTTPSConnectionPool(host='www.fpi.nsdl.co.in', port=443): Read timed out.
Trying: https://www.fpi.nsdl.co.in/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_AUG312025.html
  ✓ Success! Got 29 rows

✓ Total downloaded: 5 reports


In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

def download_nsdl_with_browser():
    """Use Selenium to automate browser downloads"""
    
    # Setup Chrome driver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in background
    driver = webdriver.Chrome(options=options)
    
    # Go to selection page
    driver.get("https://www.fpi.nsdl.co.in/web/Reports/FPI_Fortnightly_Selection.aspx")
    
    # Find all report links
    links = driver.find_elements(By.TAG_NAME, "a")
    
    report_urls = []
    for link in links:
        href = link.get_attribute('href')
        if href and 'FII' in href:
            report_urls.append(href)
    
    print(f"Found {len(report_urls)} report links")
    
    # Download each report
    for i, url in enumerate(report_urls[:10]):  # First 10 for testing
        print(f"Downloading {i+1}/{len(report_urls)}: {url}")
        driver.get(url)
        time.sleep(3)
        
        # Get page content
        page_source = driver.page_source
        
        # Parse tables
        tables = pd.read_html(page_source)
        if tables:
            tables[0].to_csv(f'nsdl_report_{i+1}.csv', index=False)
            print(f"  ✓ Saved nsdl_report_{i+1}.csv")
    
    driver.quit()

# Requires: pip install selenium
download_nsdl_with_browser()


WebDriverException: Message: unknown error: net::ERR_CONNECTION_RESET
  (Session info: chrome=142.0.7444.176)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff65176a235
	0x7ff6514c2630
	0x7ff6512516dd
	0x7ff65124e228
	0x7ff65123eba9
	0x7ff6512409d7
	0x7ff65123f160
	0x7ff65123e917
	0x7ff65123e5db
	0x7ff65123c145
	0x7ff65123c9dc
	0x7ff6512559ea
	0x7ff6512fc8be
	0x7ff6512d2b0a
	0x7ff6512fbaba
	0x7ff65129b0ed
	0x7ff65129bf63
	0x7ff651795d60
	0x7ff65178fe8a
	0x7ff6517b1005
	0x7ff6514dd71e
	0x7ff6514e4e1f
	0x7ff6514cb7c4
	0x7ff6514cb97f
	0x7ff6514b18e8
	0x7ffcc6ebe8d7
	0x7ffcc856c53c


In [14]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thedevastator/us-renewable-energy-programs")

print("Path to dataset files:", path)

100%|██████████| 6.47M/6.47M [00:02<00:00, 2.97MB/s]

Extracting files...


Path to dataset files: C:\Users\Soumya Pandey\.cache\kagglehub\datasets\thedevastator\us-renewable-energy-programs\versions\2


In [13]:
pip install kagglehub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [15]:
# green_stock_downloader.py
# Run this script on your local machine after: pip install yfinance pandas numpy

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import time

print("="*80)
print("GREEN STOCK UNIVERSE DATA COLLECTION")
print("="*80)

# ============================================================================
# 1. DEFINE COMPLETE GREEN STOCK UNIVERSE
# ============================================================================

print("\n[1] Defining stock universe...")

# Clean Energy ETFs
clean_energy_etfs = {
    'ICLN': 'iShares Global Clean Energy ETF',
    'TAN': 'Invesco Solar ETF',
    'QCLN': 'First Trust NASDAQ Clean Edge Green Energy',
    'PBW': 'Invesco WilderHill Clean Energy ETF',
    'ACES': 'ALPS Clean Energy ETF',
    'ERTH': 'Invesco MSCI Sustainable Future ETF',
    'FAN': 'First Trust Global Wind Energy ETF',
    'SMOG': 'VanEck Low Carbon Energy ETF'
}

# Indian Renewable Energy Stocks (NSE)
indian_green_stocks = {
    'ADANIGREEN.NS': 'Adani Green Energy',
    'TATAPOWER.NS': 'Tata Power',
    'SUZLON.NS': 'Suzlon Energy',
    'POWERGRID.NS': 'Power Grid Corporation',
    'NTPC.NS': 'NTPC',
    'NHPC.NS': 'NHPC Limited',
    'SJVN.NS': 'SJVN Limited',
    'JINDALSAW.NS': 'Jindal Saw',
    'ORIENTGREEN.NS': 'Orient Green Power',
    'WEBSOL.NS': 'Websol Energy System'
}

# Global Renewable Energy Leaders
global_green_stocks = {
    # US Solar & Storage
    'FSLR': 'First Solar',
    'ENPH': 'Enphase Energy',
    'SEDG': 'SolarEdge Technologies',
    'RUN': 'Sunrun',
    'NOVA': 'Sunnova Energy',
    'MAXN': 'Maxeon Solar Technologies',
    
    # US Utilities & Developers
    'NEE': 'NextEra Energy',
    'AES': 'AES Corporation',
    'DUK': 'Duke Energy',
    'SO': 'Southern Company',
    
    # Yieldcos & MLPs
    'BEP': 'Brookfield Renewable Partners',
    'BEPC': 'Brookfield Renewable Corporation',
    'CWEN': 'Clearway Energy',
    'NEP': 'NextEra Energy Partners',
    
    # Chinese Solar (ADRs)
    'JKS': 'JinkoSolar',
    'DQ': 'Daqo New Energy',
    'CSIQ': 'Canadian Solar',
    'SOL': 'ReneSola',
    
    # European (ADRs where available)
    'DNNGY': 'Orsted ADR',
    'IBDRY': 'Iberdrola ADR',
    
    # Equipment & Tech
    'PLUG': 'Plug Power (hydrogen)',
    'BE': 'Bloom Energy (fuel cells)',
    'BLNK': 'Blink Charging',
    'CHPT': 'ChargePoint'
}

# Combine all
all_stocks = {
    **clean_energy_etfs,
    **indian_green_stocks,
    **global_green_stocks
}

print(f"Total universe: {len(all_stocks)} securities")
print(f"  - ETFs: {len(clean_energy_etfs)}")
print(f"  - Indian stocks: {len(indian_green_stocks)}")
print(f"  - Global stocks: {len(global_green_stocks)}")

# ============================================================================
# 2. DOWNLOAD DAILY PRICE DATA (2020-2025)
# ============================================================================

print("\n[2] Downloading daily price data (2020-2025)...")
print("This will take 5-10 minutes...\n")

start_date = '2020-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

def download_stock_data(ticker, name, start, end):
    """Download daily OHLCV data for a ticker"""
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(start=start, end=end, actions=True)
        
        if len(df) > 0:
            df['ticker'] = ticker
            df['name'] = name
            df.reset_index(inplace=True)
            
            # Standardize columns
            df.columns = [col.lower().replace(' ', '_') for col in df.columns]
            
            # Calculate returns
            df['return'] = df['close'].pct_change()
            df['log_return'] = np.log(df['close'] / df['close'].shift(1))
            
            return df
        return None
            
    except Exception as e:
        print(f"  ✗ {ticker}: {str(e)[:60]}")
        return None

# Download all stocks
all_price_data = []
successful = 0
failed = 0
failed_tickers = []

for i, (ticker, name) in enumerate(all_stocks.items(), 1):
    print(f"[{i}/{len(all_stocks)}] {ticker:20} {name[:35]:35}", end=' ')
    
    df = download_stock_data(ticker, name, start_date, end_date)
    
    if df is not None and len(df) > 0:
        all_price_data.append(df)
        successful += 1
        print(f"✓ {len(df):4} days")
    else:
        failed += 1
        failed_tickers.append(ticker)
        print(f"✗")
    
    # Rate limiting (be nice to Yahoo)
    time.sleep(0.25)

print(f"\n✓ Successfully downloaded: {successful}/{len(all_stocks)} securities")
print(f"✗ Failed: {failed}/{len(all_stocks)} securities")

if failed_tickers:
    print(f"\nFailed tickers: {', '.join(failed_tickers)}")

# ============================================================================
# 3. COMBINE AND SAVE DATA
# ============================================================================

if all_price_data:
    print("\n[3] Processing and saving data...")
    
    # Combine all dataframes
    df_all_prices = pd.concat(all_price_data, ignore_index=True)
    df_all_prices = df_all_prices.sort_values(['ticker', 'date'])
    
    # Save main dataset
    df_all_prices.to_csv('green_stocks_daily_prices_2020_2025.csv', index=False)
    print(f"✓ Saved: green_stocks_daily_prices_2020_2025.csv")
    print(f"  Rows: {len(df_all_prices):,}")
    print(f"  Date range: {df_all_prices['date'].min()} to {df_all_prices['date'].max()}")
    
    # Create separate files by category
    
    # 1. ETFs only
    etf_tickers = list(clean_energy_etfs.keys())
    df_etfs = df_all_prices[df_all_prices['ticker'].isin(etf_tickers)]
    df_etfs.to_csv('clean_energy_etfs.csv', index=False)
    print(f"✓ Saved: clean_energy_etfs.csv ({len(df_etfs):,} rows)")
    
    # 2. Indian stocks only
    indian_tickers = list(indian_green_stocks.keys())
    df_indian = df_all_prices[df_all_prices['ticker'].isin(indian_tickers)]
    df_indian.to_csv('indian_green_stocks.csv', index=False)
    print(f"✓ Saved: indian_green_stocks.csv ({len(df_indian):,} rows)")
    
    # 3. Global stocks only
    global_tickers = list(global_green_stocks.keys())
    df_global = df_all_prices[df_all_prices['ticker'].isin(global_tickers)]
    df_global.to_csv('global_green_stocks.csv', index=False)
    print(f"✓ Saved: global_green_stocks.csv ({len(df_global):,} rows)")
    
    # ========================================================================
    # 4. CREATE SUMMARY STATISTICS
    # ========================================================================
    
    print("\n[4] Creating summary statistics...")
    
    summary = df_all_prices.groupby(['ticker', 'name']).agg({
        'close': ['first', 'last', 'mean', 'std', 'min', 'max'],
        'return': ['mean', 'std', 'min', 'max'],
        'volume': 'mean',
        'date': ['min', 'max', 'count']
    }).round(4)
    
    summary.columns = ['_'.join(col).strip() for col in summary.columns.values]
    summary.reset_index(inplace=True)
    
    # Calculate annualized metrics
    summary['total_return_pct'] = ((summary['close_last'] / summary['close_first']) - 1) * 100
    summary['annualized_return_pct'] = summary['return_mean'] * 252 * 100
    summary['annualized_volatility_pct'] = summary['return_std'] * np.sqrt(252) * 100
    summary['sharpe_ratio'] = summary['annualized_return_pct'] / summary['annualized_volatility_pct']
    
    # Add category labels
    def categorize(ticker):
        if ticker in clean_energy_etfs:
            return 'ETF'
        elif ticker in indian_green_stocks:
            return 'India'
        else:
            return 'Global'
    
    summary['category'] = summary['ticker'].apply(categorize)
    
    summary.to_csv('green_stocks_summary_statistics.csv', index=False)
    print(f"✓ Saved: green_stocks_summary_statistics.csv")
    
    # ========================================================================
    # 5. CREATE METADATA FILE
    # ========================================================================
    
    print("\n[5] Creating metadata file...")
    
    universe_metadata = []
    for ticker, name in all_stocks.items():
        ticker_data = df_all_prices[df_all_prices['ticker'] == ticker]
        
        if len(ticker_data) > 0:
            metadata = {
                'ticker': ticker,
                'name': name,
                'category': categorize(ticker),
                'data_available': 'Yes',
                'start_date': ticker_data['date'].min(),
                'end_date': ticker_data['date'].max(),
                'trading_days': len(ticker_data),
                'mean_volume': ticker_data['volume'].mean(),
                'latest_price': ticker_data['close'].iloc[-1]
            }
        else:
            metadata = {
                'ticker': ticker,
                'name': name,
                'category': categorize(ticker),
                'data_available': 'No',
                'start_date': None,
                'end_date': None,
                'trading_days': 0,
                'mean_volume': 0,
                'latest_price': None
            }
        
        universe_metadata.append(metadata)
    
    df_universe = pd.DataFrame(universe_metadata)
    df_universe.to_csv('green_stocks_universe_metadata.csv', index=False)
    print(f"✓ Saved: green_stocks_universe_metadata.csv")
    
    # ========================================================================
    # 6. DISPLAY SUMMARY
    # ========================================================================
    
    print("\n" + "="*80)
    print("DATA COLLECTION COMPLETE")
    print("="*80)
    
    print("\nFiles created:")
    print("  1. green_stocks_daily_prices_2020_2025.csv - Main dataset (all securities)")
    print("  2. clean_energy_etfs.csv - ETFs only")
    print("  3. indian_green_stocks.csv - Indian stocks only")
    print("  4. global_green_stocks.csv - Global stocks only")
    print("  5. green_stocks_summary_statistics.csv - Performance metrics")
    print("  6. green_stocks_universe_metadata.csv - Coverage and metadata")
    
    print("\nTop 10 Performers (Total Return %, 2020-2025):")
    print("-"*80)
    top10 = summary.nlargest(10, 'total_return_pct')[['ticker', 'name', 'total_return_pct', 'annualized_return_pct', 'annualized_volatility_pct', 'sharpe_ratio']]
    print(top10.to_string(index=False))
    
    print("\nCategory Breakdown:")
    print("-"*80)
    cat_summary = df_universe.groupby('category').agg({
        'data_available': lambda x: (x == 'Yes').sum(),
        'ticker': 'count'
    })
    cat_summary.columns = ['Downloaded', 'Total']
    cat_summary['Success_Rate'] = (cat_summary['Downloaded'] / cat_summary['Total'] * 100).round(1)
    print(cat_summary)
    
else:
    print("\n✗ No data downloaded. Check internet connection and ticker symbols.")

print("\n" + "="*80)
print("NEXT STEPS:")
print("  1. Review green_stocks_summary_statistics.csv for data quality")
print("  2. Download market indices (NIFTY 50, S&P 500) using similar code")
print("  3. Merge with your policy events and FPI flow data")
print("="*80)


GREEN STOCK UNIVERSE DATA COLLECTION

[1] Defining stock universe...
Total universe: 42 securities
  - ETFs: 8
  - Indian stocks: 10
  - Global stocks: 24

[2] Downloading daily price data (2020-2025)...
This will take 5-10 minutes...

[1/42] ICLN                 iShares Global Clean Energy ETF     

Failed to get ticker 'ICLN' reason: Failed to perform, curl: (35) Recv failure: Connection was reset. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.


  ✗ ICLN: Failed to perform, curl: (35) Recv failure: Connection was r
✗


Failed to get ticker 'TAN' reason: Failed to perform, curl: (35) Recv failure: Connection was reset. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.


[2/42] TAN                  Invesco Solar ETF                     ✗ TAN: Failed to perform, curl: (35) Recv failure: Connection was r
✗


...

✗
[19/42] FSLR                 First Solar                         ✓ 1490 days
[20/42] ENPH                 Enphase Energy                      ✓ 1490 days
[21/42] SEDG                 SolarEdge Technologies              ✓ 1490 days
[22/42] RUN                  Sunrun                              ✓ 1490 days
[23/42] NOVA                 Sunnova Energy                      

$NOVA: possibly delisted; no timezone found


✗
[24/42] MAXN                 Maxeon Solar Technologies           ✓ 1326 days
[25/42] NEE                  NextEra Energy                      ✓ 1490 days
[26/42] AES                  AES Corporation                     ✓ 1490 days
[27/42] DUK                  Duke Energy                         ✓ 1490 days
[28/42] SO                   Southern Company                    ✓ 1490 days
[29/42] BEP                  Brookfield Renewable Partners       ✓ 1490 days
[30/42] BEPC                 Brookfield Renewable Corporation    ✓ 1349 days
[31/42] CWEN                 Clearway Energy                     ✓ 1490 days
[32/42] NEP                  NextEra Energy Partners             

$NEP: possibly delisted; no timezone found


✗
[33/42] JKS                  JinkoSolar                          ✓ 1490 days
[34/42] DQ                   Daqo New Energy                     ✓ 1490 days
[35/42] CSIQ                 Canadian Solar                      ✓ 1490 days
[36/42] SOL                  ReneSola                            ✓ 1490 days
[37/42] DNNGY                Orsted ADR                          ✓ 1490 days
[38/42] IBDRY                Iberdrola ADR                       ✓ 1490 days
[39/42] PLUG                 Plug Power (hydrogen)               ✓ 1490 days
[40/42] BE                   Bloom Energy (fuel cells)           ✓ 1490 days
[41/42] BLNK                 Blink Charging                      ✓ 1490 days
[42/42] CHPT                 ChargePoint                         ✓ 1490 days

✓ Successfully downloaded: 29/42 securities
✗ Failed: 13/42 securities

Failed tickers: ICLN, TAN, QCLN, PBW, ACES, ERTH, FAN, SMOG, ADANIGREEN.NS, ORIENTGREEN.NS, WEBSOL.NS, NOVA, NEP

[3] Processing and saving data...
✓ Saved

In [16]:

# control_variables_downloader.py
# Download market indices, commodities, FX, and volatility data for research controls
# Run after: pip install yfinance pandas numpy

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import time

print("="*80)
print("CONTROL VARIABLES DATA COLLECTION FOR FPI-POLICY RESEARCH")
print("="*80)

# ============================================================================
# 1. DEFINE CONTROL VARIABLES
# ============================================================================

print("\n[1] Defining control variables...")

# Market Indices
market_indices = {
    '^NSEI': 'NIFTY 50 (India)',
    '^GSPC': 'S&P 500 (US)',
    '^STOXX50E': 'EURO STOXX 50 (Europe)',
    '^N225': 'Nikkei 225 (Japan)',
    '^FTSE': 'FTSE 100 (UK)',
    '^DJI': 'Dow Jones Industrial Average',
    '^IXIC': 'NASDAQ Composite',
    '^RUT': 'Russell 2000'
}

# Sector Indices (for sector controls)
sector_indices = {
    'XLU': 'Utilities Select Sector SPDR (US)',
    'XLE': 'Energy Select Sector SPDR (US)',
    'XLI': 'Industrials Select Sector SPDR (US)'
}

# Commodities
commodities = {
    'BZ=F': 'Brent Crude Oil',
    'CL=F': 'WTI Crude Oil',
    'NG=F': 'Natural Gas',
    'GC=F': 'Gold',
    'SI=F': 'Silver'
}

# Foreign Exchange
fx_rates = {
    'INRUSD=X': 'USD/INR',
    'EURUSD=X': 'EUR/USD',
    'GBPUSD=X': 'GBP/USD',
    'JPYUSD=X': 'JPY/USD',
    'CNYUSD=X': 'CNY/USD'
}

# Volatility Indices
volatility_indices = {
    '^VIX': 'CBOE Volatility Index (US)',
    '^VXEEM': 'Emerging Markets Volatility Index',
}

# Indian specific indices
indian_indices = {
    '^NSEBANK': 'NIFTY Bank Index',
    '^CNXIT': 'NIFTY IT Index',
    '^CNXENERGY': 'NIFTY Energy Index',
    'NIFTYSMLCAP100.NS': 'NIFTY Smallcap 100'
}

# Combine all
all_controls = {
    **market_indices,
    **sector_indices,
    **commodities,
    **fx_rates,
    **volatility_indices,
    **indian_indices
}

print(f"\nTotal control variables: {len(all_controls)}")
print(f"  - Market indices: {len(market_indices)}")
print(f"  - Sector indices: {len(sector_indices)}")
print(f"  - Commodities: {len(commodities)}")
print(f"  - FX rates: {len(fx_rates)}")
print(f"  - Volatility: {len(volatility_indices)}")
print(f"  - Indian indices: {len(indian_indices)}")

# ============================================================================
# 2. DOWNLOAD DAILY DATA (2020-2025)
# ============================================================================

print("\n[2] Downloading daily data (2020-2025)...")
print("This will take 3-5 minutes...\n")

start_date = '2020-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

def download_control_data(ticker, name, start, end):
    """Download daily price/level data for control variable"""
    try:
        data = yf.Ticker(ticker)
        df = data.history(start=start, end=end)

        if len(df) > 0:
            df['ticker'] = ticker
            df['name'] = name
            df.reset_index(inplace=True)

            # Standardize columns
            df.columns = [col.lower().replace(' ', '_') for col in df.columns]

            # Calculate returns and changes
            df['return'] = df['close'].pct_change()
            df['log_return'] = np.log(df['close'] / df['close'].shift(1))
            df['price_change'] = df['close'] - df['close'].shift(1)

            # For indices/FX, calculate rolling volatility
            df['volatility_20d'] = df['return'].rolling(window=20).std() * np.sqrt(252)
            df['volatility_60d'] = df['return'].rolling(window=60).std() * np.sqrt(252)

            return df
        return None

    except Exception as e:
        print(f"  ✗ {ticker}: {str(e)[:60]}")
        return None

# Download all controls
all_control_data = []
successful = 0
failed = 0
failed_tickers = []

for i, (ticker, name) in enumerate(all_controls.items(), 1):
    print(f"[{i}/{len(all_controls)}] {ticker:20} {name[:40]:40}", end=' ')

    df = download_control_data(ticker, name, start_date, end_date)

    if df is not None and len(df) > 0:
        all_control_data.append(df)
        successful += 1
        print(f"✓ {len(df):4} days")
    else:
        failed += 1
        failed_tickers.append((ticker, name))
        print(f"✗")

    time.sleep(0.2)  # Rate limiting

print(f"\n✓ Successfully downloaded: {successful}/{len(all_controls)} variables")
print(f"✗ Failed: {failed}/{len(all_controls)} variables")

if failed_tickers:
    print(f"\nFailed downloads:")
    for ticker, name in failed_tickers:
        print(f"  - {ticker}: {name}")

# ============================================================================
# 3. PROCESS AND SAVE DATA
# ============================================================================

if all_control_data:
    print("\n[3] Processing and saving data...")

    # Combine all data
    df_all_controls = pd.concat(all_control_data, ignore_index=True)
    df_all_controls = df_all_controls.sort_values(['ticker', 'date'])

    # Save main dataset
    df_all_controls.to_csv('control_variables_daily_2020_2025.csv', index=False)
    print(f"✓ Saved: control_variables_daily_2020_2025.csv")
    print(f"  Rows: {len(df_all_controls):,}")
    print(f"  Variables: {df_all_controls['ticker'].nunique()}")
    print(f"  Date range: {df_all_controls['date'].min()} to {df_all_controls['date'].max()}")

    # Save separate files by category

    # 1. Market indices only
    market_tickers = list(market_indices.keys())
    df_markets = df_all_controls[df_all_controls['ticker'].isin(market_tickers)]
    df_markets.to_csv('market_indices_daily.csv', index=False)
    print(f"✓ Saved: market_indices_daily.csv ({len(df_markets):,} rows)")

    # 2. Commodities only
    commodity_tickers = list(commodities.keys())
    df_commodities = df_all_controls[df_all_controls['ticker'].isin(commodity_tickers)]
    df_commodities.to_csv('commodities_daily.csv', index=False)
    print(f"✓ Saved: commodities_daily.csv ({len(df_commodities):,} rows)")

    # 3. FX rates only
    fx_tickers = list(fx_rates.keys())
    df_fx = df_all_controls[df_all_controls['ticker'].isin(fx_tickers)]
    df_fx.to_csv('fx_rates_daily.csv', index=False)
    print(f"✓ Saved: fx_rates_daily.csv ({len(df_fx):,} rows)")

    # 4. Volatility indices only
    vol_tickers = list(volatility_indices.keys())
    df_vol = df_all_controls[df_all_controls['ticker'].isin(vol_tickers)]
    df_vol.to_csv('volatility_indices_daily.csv', index=False)
    print(f"✓ Saved: volatility_indices_daily.csv ({len(df_vol):,} rows)")

    # 5. Indian indices only
    indian_tickers = list(indian_indices.keys())
    df_indian_idx = df_all_controls[df_all_controls['ticker'].isin(indian_tickers)]
    df_indian_idx.to_csv('indian_indices_daily.csv', index=False)
    print(f"✓ Saved: indian_indices_daily.csv ({len(df_indian_idx):,} rows)")

    # ========================================================================
    # 4. CREATE WIDE FORMAT (ONE DATE PER ROW)
    # ========================================================================

    print("\n[4] Creating wide-format (panel-ready) dataset...")

    # Pivot to wide format for easier merging with other datasets
    df_wide = df_all_controls.pivot_table(
        index='date',
        columns='ticker',
        values=['close', 'return', 'volume', 'volatility_20d']
    )

    # Flatten column names
    df_wide.columns = ['_'.join(col).strip() for col in df_wide.columns.values]
    df_wide.reset_index(inplace=True)

    # Rename columns to be more readable
    df_wide.columns = df_wide.columns.str.replace('^', '', regex=False)
    df_wide.columns = df_wide.columns.str.replace('=', '_', regex=False)
    df_wide.columns = df_wide.columns.str.replace('.', '_', regex=False)

    df_wide.to_csv('control_variables_wide_format.csv', index=False)
    print(f"✓ Saved: control_variables_wide_format.csv")
    print(f"  Rows: {len(df_wide):,} (one row per date)")
    print(f"  Columns: {len(df_wide.columns)} (date + 4 metrics × {df_all_controls['ticker'].nunique()} variables)")

    # ========================================================================
    # 5. CREATE SUMMARY STATISTICS
    # ========================================================================

    print("\n[5] Creating summary statistics...")

    summary_stats = df_all_controls.groupby(['ticker', 'name']).agg({
        'close': ['first', 'last', 'mean', 'std', 'min', 'max'],
        'return': ['mean', 'std', 'min', 'max', 'skew'],
        'volume': 'mean',
        'date': ['min', 'max', 'count']
    }).round(4)

    summary_stats.columns = ['_'.join(col).strip() for col in summary_stats.columns.values]
    summary_stats.reset_index(inplace=True)

    # Calculate additional metrics
    summary_stats['total_change_pct'] = ((summary_stats['close_last'] / summary_stats['close_first']) - 1) * 100
    summary_stats['annualized_return_pct'] = summary_stats['return_mean'] * 252 * 100
    summary_stats['annualized_volatility_pct'] = summary_stats['return_std'] * np.sqrt(252) * 100

    # Add category
    def categorize_control(ticker):
        if ticker in market_indices:
            return 'Market Index'
        elif ticker in sector_indices:
            return 'Sector Index'
        elif ticker in commodities:
            return 'Commodity'
        elif ticker in fx_rates:
            return 'FX Rate'
        elif ticker in volatility_indices:
            return 'Volatility'
        elif ticker in indian_indices:
            return 'India Index'
        return 'Other'

    summary_stats['category'] = summary_stats['ticker'].apply(categorize_control)

    summary_stats.to_csv('control_variables_summary.csv', index=False)
    print(f"✓ Saved: control_variables_summary.csv")

    # ========================================================================
    # 6. CREATE CORRELATION MATRIX
    # ========================================================================

    print("\n[6] Creating correlation matrix...")

    # Calculate correlations between control variables
    correlation_data = df_all_controls.pivot_table(
        index='date',
        columns='ticker',
        values='return'
    )

    corr_matrix = correlation_data.corr().round(3)
    corr_matrix.to_csv('control_variables_correlation_matrix.csv')
    print(f"✓ Saved: control_variables_correlation_matrix.csv")

    # ========================================================================
    # 7. DISPLAY RESULTS
    # ========================================================================

    print("\n" + "="*80)
    print("DATA COLLECTION COMPLETE")
    print("="*80)

    print("\nFiles created:")
    print("  1. control_variables_daily_2020_2025.csv - Main dataset (long format)")
    print("  2. control_variables_wide_format.csv - Wide format (ready for merging)")
    print("  3. market_indices_daily.csv - Market indices only")
    print("  4. commodities_daily.csv - Commodity prices only")
    print("  5. fx_rates_daily.csv - FX rates only")
    print("  6. volatility_indices_daily.csv - Volatility indices only")
    print("  7. indian_indices_daily.csv - Indian indices only")
    print("  8. control_variables_summary.csv - Summary statistics")
    print("  9. control_variables_correlation_matrix.csv - Correlation matrix")

    print("\nKey Variables Summary:")
    print("-"*80)

    # Show key statistics for critical variables
    key_vars = ['^NSEI', '^GSPC', 'BZ=F', 'INRUSD=X', '^VIX']
    key_summary = summary_stats[summary_stats['ticker'].isin(key_vars)][
        ['ticker', 'name', 'close_first', 'close_last', 'total_change_pct', 
         'annualized_return_pct', 'annualized_volatility_pct']
    ]

    if len(key_summary) > 0:
        print(key_summary.to_string(index=False))

    print("\nCategory Breakdown:")
    print("-"*80)
    cat_counts = summary_stats.groupby('category').agg({
        'ticker': 'count',
        'date_count': 'mean'
    })
    cat_counts.columns = ['Count', 'Avg_Trading_Days']
    print(cat_counts)

    print("\nData Coverage:")
    print("-"*80)
    coverage_stats = {
        'Start Date': df_all_controls['date'].min(),
        'End Date': df_all_controls['date'].max(),
        'Total Days': (df_all_controls['date'].max() - df_all_controls['date'].min()).days,
        'Variables Downloaded': df_all_controls['ticker'].nunique(),
        'Total Observations': len(df_all_controls)
    }
    for key, value in coverage_stats.items():
        print(f"  {key:25}: {value}")

    # ========================================================================
    # 8. CREATE REGRESSION-READY DATASET
    # ========================================================================

    print("\n[8] Creating regression-ready dataset...")

    # Select key variables for typical regression specification
    regression_vars = {
        '^NSEI': 'nifty50_return',
        '^GSPC': 'sp500_return',
        'BZ=F': 'brent_return',
        'INRUSD=X': 'usdinr_change',
        '^VIX': 'vix_level'
    }

    df_regression = pd.DataFrame({'date': df_wide['date']})

    for ticker, var_name in regression_vars.items():
        clean_ticker = ticker.replace('^', '').replace('=', '_').replace('.', '_')

        # Try to find the column (handle various naming conventions)
        close_col = f'close_{clean_ticker}'
        return_col = f'return_{clean_ticker}'

        if close_col in df_wide.columns:
            if 'vix' in var_name.lower():
                df_regression[var_name] = df_wide[close_col]  # VIX level, not return
            else:
                df_regression[var_name] = df_wide.get(return_col, df_wide[close_col].pct_change())

    df_regression.to_csv('control_variables_regression_ready.csv', index=False)
    print(f"✓ Saved: control_variables_regression_ready.csv")
    print(f"  Contains key control variables in clean format for regression")

else:
    print("\n✗ No data downloaded. Check internet connection and ticker symbols.")

print("\n" + "="*80)
print("NEXT STEPS:")
print("  1. Review control_variables_summary.csv for data quality")
print("  2. Use control_variables_wide_format.csv to merge with your event data")
print("  3. Use control_variables_regression_ready.csv for your panel regressions")
print("="*80)

CONTROL VARIABLES DATA COLLECTION FOR FPI-POLICY RESEARCH

[1] Defining control variables...

Total control variables: 27
  - Market indices: 8
  - Sector indices: 3
  - Commodities: 5
  - FX rates: 5
  - Volatility: 2
  - Indian indices: 4

[2] Downloading daily data (2020-2025)...
This will take 3-5 minutes...

[1/27] ^NSEI                NIFTY 50 (India)                         ✓ 1468 days
[2/27] ^GSPC                S&P 500 (US)                             ✓ 1490 days
[3/27] ^STOXX50E            EURO STOXX 50 (Europe)                   ✓ 1495 days
[4/27] ^N225                Nikkei 225 (Japan)                       ✓ 1447 days
[5/27] ^FTSE                FTSE 100 (UK)                            ✓ 1497 days
[6/27] ^DJI                 Dow Jones Industrial Average             ✓ 1490 days
[7/27] ^IXIC                NASDAQ Composite                         ✓ 1490 days
[8/27] ^RUT                 Russell 2000                             ✓ 1490 days
[9/27] XLU                  Utilities

c:\ProgramData\anaconda3\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


✓ 1486 days
[14/27] NG=F                 Natural Gas                              ✓ 1487 days
[15/27] GC=F                 Gold                                     ✓ 1486 days
[16/27] SI=F                 Silver                                   ✓ 1486 days
[17/27] INRUSD=X             USD/INR                                  ✓ 1544 days
[18/27] EURUSD=X             EUR/USD                                  ✓ 1544 days
[19/27] GBPUSD=X             GBP/USD                                  ✓ 1544 days
[20/27] JPYUSD=X             JPY/USD                                  ✓ 1544 days
[21/27] CNYUSD=X             CNY/USD                                  ✓ 1544 days
[22/27] ^VIX                 CBOE Volatility Index (US)               ✓ 1490 days
[23/27] ^VXEEM               Emerging Markets Volatility Index        ✓    1 days
[24/27] ^NSEBANK             NIFTY Bank Index                         ✓ 1461 days
[25/27] ^CNXIT               NIFTY IT Index                           ✓ 1461 days
[26/

$NIFTYSMLCAP100.NS: possibly delisted; no timezone found


✗

✓ Successfully downloaded: 26/27 variables
✗ Failed: 1/27 variables

Failed downloads:
  - NIFTYSMLCAP100.NS: NIFTY Smallcap 100

[3] Processing and saving data...
✓ Saved: control_variables_daily_2020_2025.csv
  Rows: 37,355
  Variables: 26
  Date range: 2020-01-01 00:00:00+05:30 to 2025-12-04 00:00:00-06:00
✓ Saved: market_indices_daily.csv (11,867 rows)
✓ Saved: commodities_daily.csv (7,432 rows)
✓ Saved: fx_rates_daily.csv (7,720 rows)
✓ Saved: volatility_indices_daily.csv (1,491 rows)
✓ Saved: indian_indices_daily.csv (4,375 rows)

[4] Creating wide-format (panel-ready) dataset...
✓ Saved: control_variables_wide_format.csv
  Rows: 8,937 (one row per date)
  Columns: 103 (date + 4 metrics × 26 variables)

[5] Creating summary statistics...
✓ Saved: control_variables_summary.csv

[6] Creating correlation matrix...
✓ Saved: control_variables_correlation_matrix.csv

DATA COLLECTION COMPLETE

Files created:
  1. control_variables_daily_2020_2025.csv - Main dataset (long format)
  2.

C:\Users\Soumya Pandey\AppData\Local\Temp\ipykernel_16180\2633847970.py:377: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_regression[var_name] = df_wide.get(return_col, df_wide[close_col].pct_change())
C:\Users\Soumya Pandey\AppData\Local\Temp\ipykernel_16180\2633847970.py:377: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_regression[var_name] = df_wide.get(return_col, df_wide[close_col].pct_change())
C:\Users\Soumya Pandey\AppData\Local\Temp\ipykernel_16180\2633847970.py:377: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future v

In [17]:

import pandas as pd
from datetime import datetime
import os

print("="*80)
print("HISTORICAL INDIA RENEWABLE ENERGY POLICY EVENTS - COMPLETE COLLECTION")
print("="*80)

# ============================================================================
# PRE-CODED MAJOR POLICY EVENTS (2020-2024)
# Source: PIB, MNRE, Ministry of Finance official announcements
# ============================================================================

print("\n[1] Loading 13 pre-coded major policy events (2020-2024)...")

major_policy_events = [
    # 2023 - HIGH ACTIVITY YEAR
    {
        'announcement_date': '2023-01-04',
        'title': 'National Green Hydrogen Mission Approval',
        'policy_type': 'fiscal',
        'tech_focus': 'green_hydrogen',
        'description': 'Cabinet approved National Green Hydrogen Mission with Rs 19,744 crore outlay, targeting 5 MMT annual production by 2030',
        'importance': 'very_high',
        'source': 'PIB Press Release',
        'url': 'https://www.pib.gov.in/PressReleaseIframePage.aspx?PRID=2039091',
        'budget_amount_cr': 19744,
        'target': '5 MMT green hydrogen by 2030',
        'notes': 'Major fiscal commitment to green hydrogen'
    },
    {
        'announcement_date': '2023-06-19',
        'title': 'National Green Hydrogen Mission Guidelines Released',
        'policy_type': 'operational',
        'tech_focus': 'green_hydrogen',
        'description': 'Detailed implementation guidelines for NGHM released by MNRE',
        'importance': 'high',
        'source': 'MNRE',
        'url': 'https://mnre.gov.in',
        'budget_amount_cr': None,
        'target': None,
        'notes': 'Operationalization of NGHM'
    },
    {
        'announcement_date': '2023-08-01',
        'title': 'ALMM Order for Solar PV Modules (Initial)',
        'policy_type': 'regulatory',
        'tech_focus': 'solar_pv',
        'description': 'Approved List of Models and Manufacturers order issued, mandating use of listed modules in govt schemes',
        'importance': 'very_high',
        'source': 'MNRE',
        'url': 'https://mnre.gov.in',
        'budget_amount_cr': None,
        'target': 'Mandatory for govt-backed projects',
        'notes': 'Major supply-side regulatory intervention'
    },
    
    # 2022 - PLI AND TARIFF CHANGES
    {
        'announcement_date': '2022-09-29',
        'title': 'PLI Scheme for Solar Modules - Tranche II Launch',
        'policy_type': 'fiscal',
        'tech_focus': 'solar_manufacturing',
        'description': 'Second tranche of PLI scheme launched with Rs 13,938 crore allocation for 39,600 MW capacity',
        'importance': 'very_high',
        'source': 'MNRE/PIB',
        'url': 'https://mnre.gov.in',
        'budget_amount_cr': 13938,
        'target': '39,600 MW capacity (3 basket structure)',
        'notes': 'Major manufacturing incentive'
    },
    {
        'announcement_date': '2022-06-01',
        'title': 'Basic Customs Duty on Solar Modules and Cells',
        'policy_type': 'regulatory',
        'tech_focus': 'solar',
        'description': 'BCD of 40% on modules and 25% on cells introduced to boost domestic manufacturing',
        'importance': 'very_high',
        'source': 'Ministry of Finance',
        'url': 'https://www.cbic.gov.in',
        'budget_amount_cr': None,
        'target': 'Import substitution',
        'notes': 'Protectionist measure to boost domestic industry'
    },
    {
        'announcement_date': '2022-02-01',
        'title': 'Union Budget 2022-23: Renewable Energy Announcements',
        'policy_type': 'fiscal',
        'tech_focus': 'renewable_general',
        'description': 'Union Budget allocated Rs 20,700 crore for renewable energy sector',
        'importance': 'high',
        'source': 'Ministry of Finance',
        'url': 'https://www.indiabudget.gov.in',
        'budget_amount_cr': 20700,
        'target': 'Annual sector allocation',
        'notes': 'Major budget support'
    },
    
    # 2021 - PLI LAUNCH AND SCHEME EXPANSION
    {
        'announcement_date': '2021-09-29',
        'title': 'PLI Scheme for Solar Modules - Tranche I Launch',
        'policy_type': 'fiscal',
        'tech_focus': 'solar_manufacturing',
        'description': 'First tranche of Production Linked Incentive scheme launched with Rs 4,500 crore for 10,000 MW',
        'importance': 'very_high',
        'source': 'MNRE',
        'url': 'https://mnre.gov.in',
        'budget_amount_cr': 4500,
        'target': '10,000 MW capacity',
        'notes': 'Landmark manufacturing incentive scheme'
    },
    {
        'announcement_date': '2021-08-12',
        'title': 'PM-KUSUM Scheme Extension and Expansion',
        'policy_type': 'operational',
        'tech_focus': 'solar_agriculture',
        'description': 'Extension and additional allocation for PM-KUSUM scheme for solar agriculture',
        'importance': 'high',
        'source': 'MNRE',
        'url': 'https://mnre.gov.in',
        'budget_amount_cr': None,
        'target': '30.8 GW by 2022',
        'notes': 'Expansion of agricultural solar'
    },
    {
        'announcement_date': '2021-02-01',
        'title': 'Union Budget 2021-22: Renewable Energy Allocation',
        'policy_type': 'fiscal',
        'tech_focus': 'renewable_general',
        'description': 'Union Budget announced Rs 15,000 crore for renewable energy sector',
        'importance': 'high',
        'source': 'Ministry of Finance',
        'url': 'https://www.indiabudget.gov.in',
        'budget_amount_cr': 15000,
        'target': 'Annual sector allocation',
        'notes': 'Increased budget support'
    },
    
    # 2020 - COVID RECOVERY AND LONG-TERM TARGETS
    {
        'announcement_date': '2020-11-18',
        'title': '100 GW Solar Target Reaffirmation',
        'policy_type': 'policy_announcement',
        'tech_focus': 'solar',
        'description': 'Government reaffirmed 100 GW solar capacity target by 2022',
        'importance': 'medium',
        'source': 'MNRE',
        'url': 'https://mnre.gov.in',
        'budget_amount_cr': None,
        'target': '100 GW by 2022',
        'notes': 'Milestone target reaffirmed'
    },
    {
        'announcement_date': '2020-10-01',
        'title': 'Solar Park Extension',
        'policy_type': 'operational',
        'tech_focus': 'solar',
        'description': 'Extension of timeline for Solar Park scheme implementation',
        'importance': 'medium',
        'source': 'MNRE',
        'url': 'https://mnre.gov.in',
        'budget_amount_cr': None,
        'target': '40 GW solar parks',
        'notes': 'Scheme extension'
    },
    {
        'announcement_date': '2020-06-15',
        'title': 'Atmanirbhar Bharat: Solar Manufacturing Push',
        'policy_type': 'policy_announcement',
        'tech_focus': 'solar_manufacturing',
        'description': 'Self-reliant India initiative included focus on solar manufacturing and renewable energy',
        'importance': 'high',
        'source': 'PMO/MNRE',
        'url': 'https://pib.gov.in',
        'budget_amount_cr': None,
        'target': 'Domestic manufacturing boost',
        'notes': 'Strategic focus on self-reliance'
    },
    {
        'announcement_date': '2020-02-01',
        'title': 'Union Budget 2020-21: Renewable Energy Allocation',
        'policy_type': 'fiscal',
        'tech_focus': 'renewable_general',
        'description': 'Union Budget allocated funds for renewable energy programs including solar, wind, and biomass',
        'importance': 'medium',
        'source': 'Ministry of Finance',
        'url': 'https://www.indiabudget.gov.in',
        'budget_amount_cr': 12000,
        'target': 'Annual sector allocation',
        'notes': 'Budget allocation baseline'
    }
]

df_major = pd.DataFrame(major_policy_events)
df_major['country'] = 'India'
df_major['jurisdiction'] = 'National'

print(f"✓ Loaded {len(df_major)} pre-coded events")
print(f"  Date range: {df_major['announcement_date'].min()} to {df_major['announcement_date'].max()}")
print(f"  Very High importance: {(df_major['importance'] == 'very_high').sum()}")
print(f"  Total fiscal value: Rs {df_major['budget_amount_cr'].sum():,.0f} crore")

# Save pre-coded events
df_major.to_csv('india_policy_events_2020_2024_precoded.csv', index=False)
print(f"✓ Saved: india_policy_events_2020_2024_precoded.csv")

# ============================================================================
# CREATE MANUAL CODING TEMPLATE
# ============================================================================

print("\n[2] Creating manual coding template...")

# Template with 10 empty rows for user to fill
manual_template = pd.DataFrame({
    'announcement_date': ['YYYY-MM-DD'] * 10,
    'title': ['[Event Title]'] * 10,
    'policy_type': ['fiscal|regulatory|operational'] * 10,
    'tech_focus': ['solar|wind|hydrogen|biomass|general'] * 10,
    'description': ['[One-sentence description]'] * 10,
    'importance': ['very_high|high|medium|low'] * 10,
    'source': ['MNRE|PIB|Finance|SECI'] * 10,
    'url': ['https://...'] * 10,
    'budget_amount_cr': [None] * 10,
    'target': ['[Numeric target or None]'] * 10,
    'notes': ['[Additional context]'] * 10,
    'country': ['India'] * 10,
    'jurisdiction': ['National'] * 10
})

manual_template.to_csv('TEMPLATE_manual_policy_coding_10events.csv', index=False)
print(f"✓ Saved: TEMPLATE_manual_policy_coding_10events.csv")

# ============================================================================
# CREATE RESEARCH GUIDE
# ============================================================================

print("\n[3] Creating detailed research guide...")

research_guide = """
================================================================================
MANUAL POLICY EVENT COLLECTION GUIDE (2020-2024)
================================================================================

OVERALL STRATEGY:
You have 13 pre-coded major events. Target 30-40 additional events to reach 50-60 total.
This requires ~3-4 hours of focused research.

================================================================================
TIER 1 SOURCES (Highest Priority - Easy to Find)
================================================================================

1. UNION BUDGETS (5 years x 2 hours each = 10 hours, but only 3-4 needed)
   
   Quick Summary Approach:
   - Go to indiabudget.gov.in and search for "renewable energy" or "MNRE"
   - Extract: New allocations, new schemes, target changes
   - Expected: 1-2 major events per year = 5-10 events total
   
   Links:
   - 2024-25: https://www.indiabudget.gov.in/budget2024-25/
   - 2023-24: https://www.indiabudget.gov.in/budget2023-24/
   - 2022-23: https://www.indiabudget.gov.in/budget2022-23/
   - 2021-22: https://www.indiabudget.gov.in/budget2021-22/
   - 2020-21: https://www.indiabudget.gov.in/budget2020-21/
   
   Things to Code:
   ✓ Any budget allocation increase/decrease
   ✓ New scheme announcements
   ✓ Target revisions (100 GW → 175 GW, etc.)
   ✓ New ministry initiatives mentioned

2. PIB PRESS RELEASES (2 hours)
   
   Search Strategy:
   - Visit: https://pib.gov.in
   - Search: "MNRE" OR "renewable energy" OR "green energy"
   - Filter by year (2020-2024)
   - Look for Cabinet approvals, major launches
   
   Things to Code:
   ✓ Cabinet Committee on Economic Affairs (CCEA) approvals
   ✓ New scheme launches
   ✓ Amendments to existing schemes
   ✓ Target or target-related announcements

3. MNRE OFFICIAL ANNOUNCEMENTS (1-2 hours)
   
   Resources:
   - Main site: https://mnre.gov.in/en/
   - Past notices: https://mnre.gov.in/en/past-notices/current-notices/
   - Year-wise achievements: https://mnre.gov.in/en/year-wise-achievement/
   
   Things to Code:
   ✓ Major program extensions
   ✓ ALMM list updates (quarterly 2023-2024)
   ✓ Quality control order changes
   ✓ Scheme guideline updates
   ✓ Large tender announcements (>1 GW)

================================================================================
TIER 2 SOURCES (Medium Priority - Specific Policies)
================================================================================

4. SECI TENDERS (1 hour)
   
   Search Strategy:
   - Visit: https://seci.co.in/tenders
   - Filter for tenders >1,000 MW
   - Note announcement dates
   
   Things to Code:
   ✓ Major tender awards (>1 GW) as policy signals
   ✓ Offshore wind tenders (new program)
   ✓ Manufacturing tenders

5. STATE GOVERNMENT POLICIES (1 hour optional)
   
   Priority States:
   - Gujarat (largest solar state)
   - Rajasthan (large renewable capacity)
   - Tamil Nadu (wind power leader)
   - Maharashtra (recent expansion)
   
   Things to Code:
   ✓ Major policy changes in top states
   ✓ State-specific targets or mandates
   ✓ State-level PLI or incentive announcements

6. ECONOMIC TIMES / BUSINESS STANDARD ARCHIVES (30 min)
   
   Search Strategy:
   - ET: https://economictimes.indiatimes.com/industry/renewables
   - BS: https://www.business-standard.com/
   - Search: "MNRE" + "policy" + year
   
   Use for: Confirmation of dates and details you found elsewhere
   DON'T code news articles directly - use them to find official source

================================================================================
QUICK REFERENCE: MAJOR EVENTS BY YEAR
================================================================================

2024 (Since Jan 1):
[ ] PM Surya Ghar campaign phase updates?
[ ] ALMM list updates (check quarterly)?
[ ] New battery storage policy?
[ ] Offshore wind progress?
Expected: 3-5 events

2023 (Complete):
[✓] National Green Hydrogen Mission (Jan 4) - PRE-CODED
[✓] ALMM Order (Aug 1) - PRE-CODED
[ ] ALMM List revisions (multiple updates)?
[ ] Solar PV cell ALMM order?
[ ] Union Budget (Feb 1)?
Expected: 3-5 additional events

2022 (Complete):
[✓] PLI Tranche II (Sep 29) - PRE-CODED
[✓] Basic Customs Duty (Jun 1) - PRE-CODED
[✓] Budget 2022-23 (Feb 1) - PRE-CODED
[ ] SECI wind tender announcements?
[ ] State-level policy changes?
Expected: 2-4 additional events

2021 (Complete):
[✓] PLI Tranche I (Sep 29) - PRE-CODED
[✓] PM-KUSUM expansion (Aug 12) - PRE-CODED
[✓] Budget 2021-22 (Feb 1) - PRE-CODED
[ ] Offshore wind policy framework?
[ ] Battery storage announcements?
Expected: 1-3 additional events

2020 (Complete):
[✓] Atmanirbhar Bharat (Jun 15) - PRE-CODED
[✓] 100 GW reaffirmation (Nov 18) - PRE-CODED
[✓] Solar Park extension (Oct 1) - PRE-CODED
[✓] Budget 2020-21 (Feb 1) - PRE-CODED
[ ] COVID-related policy adjustments?
Expected: 0-2 additional events

================================================================================
CODING INSTRUCTIONS
================================================================================

REQUIRED FIELDS:

1. announcement_date (YYYY-MM-DD)
   - Must be EXACT date from official source
   - If only month/year known, use first day of month
   - NEVER guess or approximate

2. title (max 100 characters)
   - Clear, descriptive
   - Example: "PLI Scheme for Solar Modules - Tranche II Launch"

3. policy_type (MUST be one of):
   - 'fiscal' = Money/incentive allocation (PLI, budget, subsidies)
   - 'regulatory' = Rules/mandates (ALMM, BCD, DCR)
   - 'operational' = Scheme updates/extensions (PM-KUSUM expansion)
   - 'policy_announcement' = Targets/plans (100 GW target)

4. tech_focus (MUST be one of):
   - 'solar', 'solar_pv', 'solar_manufacturing', 'solar_agriculture'
   - 'wind', 'offshore_wind'
   - 'green_hydrogen', 'hydrogen'
   - 'biomass', 'bioenergy'
   - 'renewable_general'

5. description (1 sentence, 100-200 words)
   - What was announced
   - Who announced it
   - Key numbers if applicable
   - Example: "Cabinet approved National Green Hydrogen Mission with Rs 19,744 crore outlay, targeting 5 MMT annual production by 2030"

6. importance (MUST be one of):
   - 'very_high' = Either:
     * Budget allocation >Rs 5,000 crore, OR
     * Major regulatory mandate affecting all market participants
   - 'high' = Either:
     * Budget allocation Rs 1,000-5,000 crore, OR
     * Important sector-wide mandate
   - 'medium' = Budget <Rs 1,000 crore OR target/plan announcement
   - 'low' = Technical/procedural changes

7. source (MUST be one of):
   - 'MNRE' = Ministry of New and Renewable Energy
   - 'PIB' = Press Information Bureau
   - 'Finance' = Ministry of Finance
   - 'SECI' = Solar Energy Corporation of India
   - 'State' = State government

8. url
   - Direct link to official notification/press release
   - If PDF, link to PDF
   - If PIB, link to PIB page
   - DO NOT link to news articles

9. budget_amount_cr
   - Numeric value in Rs crore (no commas)
   - None if not applicable
   - Example: 19744 (not "19,744 cr")

10. target
    - If announcement includes numeric target, enter it
    - Can be capacity (GW), production (MMT), funding amount
    - None if not applicable
    - Example: "5 MMT green hydrogen by 2030"

================================================================================
VALIDATION CHECKLIST
================================================================================

Before saving each event, verify:

☐ Date is exact (YYYY-MM-DD) from official source
☐ Title is clear and descriptive (50-100 chars)
☐ Policy type is one of: fiscal|regulatory|operational|policy_announcement
☐ Tech focus is correctly categorized
☐ Importance follows the rules (budget or mandate based)
☐ Description is 1-2 sentences, factual
☐ Source is official (not news article)
☐ URL points to official document/press release
☐ Budget is numeric in crore (no symbols)
☐ No duplicate of pre-coded events

================================================================================
EXAMPLE CODING
================================================================================

announcement_date: 2023-01-04
title: National Green Hydrogen Mission Approval
policy_type: fiscal
tech_focus: green_hydrogen
description: Cabinet approved National Green Hydrogen Mission with Rs 19,744 crore outlay, targeting 5 MMT annual production by 2030
importance: very_high
source: PIB
url: https://www.pib.gov.in/PressReleaseIframePage.aspx?PRID=2039091
budget_amount_cr: 19744
target: 5 MMT green hydrogen by 2030
notes: Major fiscal commitment to green hydrogen technology

================================================================================
ESTIMATED TIME BREAKDOWN
================================================================================

Activity                          Time      Events Expected
────────────────────────────────────────────────────────────
Review pre-coded 13 events        15 min    (already done)
Union Budgets 2020-2024           60 min    5-8 events
PIB press releases search         45 min    5-8 events
MNRE announcements & ALMM         45 min    5-10 events
SECI tender announcements         30 min    3-5 events
Date verification & validation    30 min    (quality control)
────────────────────────────────────────────────────────────
TOTAL                           3.5 hrs    Total: 50-60 events

================================================================================
FINAL OUTPUT
================================================================================

Once complete:
1. You'll have: 13 pre-coded + 30-50 manual = 50-60 total events
2. Spanning: 5 years (2020-2024)
3. Quality: Verified with official sources
4. Ready for: Event study analysis + panel regression

File to deliver:
- india_policy_events_2020_2024_precoded.csv (13 events)
- india_policy_events_2020_2024_manual.csv (your coded 30-50 events)
- Then merge both for complete dataset

================================================================================
"""

with open('POLICY_COLLECTION_RESEARCH_GUIDE.txt', 'w') as f:
    f.write(research_guide)

print(f"✓ Saved: POLICY_COLLECTION_RESEARCH_GUIDE.txt (detailed instructions)")

# ============================================================================
# DISPLAY SUMMARY
# ============================================================================

print("\n" + "="*80)
print("SETUP COMPLETE")
print("="*80)

print("\nFiles created:")
print("  1. india_policy_events_2020_2024_precoded.csv ✓ (13 events, DONE)")
print("  2. TEMPLATE_manual_policy_coding_10events.csv (fill this with your findings)")
print("  3. POLICY_COLLECTION_RESEARCH_GUIDE.txt (detailed instructions)")

print("\nNext steps:")
print("  1. Open POLICY_COLLECTION_RESEARCH_GUIDE.txt")
print("  2. Follow Tier 1 sources (3-4 hours):")
print("     - Union Budgets: 60 minutes")
print("     - PIB press releases: 45 minutes")
print("     - MNRE announcements: 45 minutes")
print("     - SECI tenders: 30 minutes")
print("  3. Fill TEMPLATE_manual_policy_coding_10events.csv")
print("  4. Merge both CSVs for complete 50-60 event dataset")

print("\nTarget outcome:")
print("  - 50-60 total events (13 precoded + 35-50 manual)")
print("  - Coverage: 2020-2024 (5 years)")
print("  - Ready for event study analysis")

print("\n" + "="*80)

HISTORICAL INDIA RENEWABLE ENERGY POLICY EVENTS - COMPLETE COLLECTION

[1] Loading 13 pre-coded major policy events (2020-2024)...
✓ Loaded 13 pre-coded events
  Date range: 2020-02-01 to 2023-08-01
  Very High importance: 5
  Total fiscal value: Rs 85,882 crore
✓ Saved: india_policy_events_2020_2024_precoded.csv

[2] Creating manual coding template...
✓ Saved: TEMPLATE_manual_policy_coding_10events.csv

[3] Creating detailed research guide...
✓ Saved: POLICY_COLLECTION_RESEARCH_GUIDE.txt (detailed instructions)

SETUP COMPLETE

Files created:
  1. india_policy_events_2020_2024_precoded.csv ✓ (13 events, DONE)
  2. TEMPLATE_manual_policy_coding_10events.csv (fill this with your findings)
  3. POLICY_COLLECTION_RESEARCH_GUIDE.txt (detailed instructions)

Next steps:
  1. Open POLICY_COLLECTION_RESEARCH_GUIDE.txt
  2. Follow Tier 1 sources (3-4 hours):
     - Union Budgets: 60 minutes
     - PIB press releases: 45 minutes
     - MNRE announcements: 45 minutes
     - SECI tenders: 30 minu

In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime

print("="*80)
print("NSDL FPI DATA DOWNLOADER (2023-2025)")
print("="*80)

# ============================================================================
# 1. DOWNLOAD MONTHLY AGGREGATED DATA (EASIEST)
# ============================================================================

print("\n[1] Downloading NSDL Monthly Data (2023-2025)...")

def scrape_nsdl_monthly_table(year):
    """Scrape monthly FPI data table for a given year"""

    url = f"https://www.fpi.nsdl.co.in/Reports/Yearwise.aspx?RptType=6"

    try:
        # Add headers to avoid blocking
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }

        # This is a simplified approach - actual implementation may need 
        # to handle ASP.NET view state and postbacks

        print(f"  Note: Visit {url} in browser and manually select year {year}")
        print(f"  Then copy the table to Excel and save as nsdl_monthly_{year}.xlsx")

        return None

    except Exception as e:
        print(f"  ✗ Error: {e}")
        return None

# For each year, user needs to manually download (ASP.NET postbacks are complex)
print("\nMANUAL DOWNLOAD REQUIRED FOR MONTHLY DATA:")
print("  1. Visit: https://www.fpi.nsdl.co.in/Reports/Yearwise.aspx?RptType=6")
print("  2. Select year: 2023, then copy table to Excel")
print("  3. Save as: nsdl_monthly_2023.xlsx")
print("  4. Repeat for 2024 and 2025")
print("  5. Takes 10 minutes total")

# ============================================================================
# 2. GENERATE FORTNIGHTLY DATE LIST
# ============================================================================

print("\n[2] Generating fortnightly dates list...")

# NSDL fortnightly dates follow a pattern: 15th and last day of each month
fortnightly_dates_2023 = []
fortnightly_dates_2024 = []
fortnightly_dates_2025 = []

months_2023 = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
months_2024 = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
months_2025 = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov']

# Pattern: Month 15, Month Last Day
month_ends = {
    'Jan': 31, 'Feb': 28, 'Mar': 31, 'Apr': 30, 'May': 31, 'Jun': 30,
    'Jul': 31, 'Aug': 31, 'Sep': 30, 'Oct': 31, 'Nov': 30, 'Dec': 31
}

print("\nFortnightly dates to download:")

print("\n2023:")
for month in months_2023:
    date1 = f"{month} 15, 2023"
    date2 = f"{month} {month_ends[month]}, 2023"
    fortnightly_dates_2023.extend([date1, date2])
    print(f"  {date1} | {date2}")

print(f"\nTotal 2023 dates: {len(fortnightly_dates_2023)}")

print("\n2024:")
for month in months_2024:
    date1 = f"{month} 15, 2024"
    date2 = f"{month} {month_ends[month]}, 2024"
    fortnightly_dates_2024.extend([date1, date2])

print(f"Total 2024 dates: {len(fortnightly_dates_2024)}")

print("\n2025 (Jan-Nov):")
for month in months_2025:
    date1 = f"{month} 15, 2025"
    date2 = f"{month} {month_ends[month]}, 2025"
    fortnightly_dates_2025.extend([date1, date2])

print(f"Total 2025 dates: {len(fortnightly_dates_2025)}")

total_dates = len(fortnightly_dates_2023) + len(fortnightly_dates_2024) + len(fortnightly_dates_2025)
print(f"\n✓ Total fortnightly dates: {total_dates}")

# Save date list for manual download
all_dates = fortnightly_dates_2023 + fortnightly_dates_2024 + fortnightly_dates_2025
df_dates = pd.DataFrame({'date': all_dates})
df_dates.to_csv('nsdl_fortnightly_dates_to_download.csv', index=False)
print(f"✓ Saved date list to: nsdl_fortnightly_dates_to_download.csv")

# ============================================================================
# 3. INSTRUCTIONS FOR MANUAL DOWNLOAD
# ============================================================================

print("\n" + "="*80)
print("DOWNLOAD INSTRUCTIONS")
print("="*80)

print("""
FORTNIGHTLY SECTOR DATA (Most Important):

1. Go to: https://www.fpi.nsdl.co.in/web/Reports/FPI_Fortnightly_Selection.aspx

2. You'll see a list of dates like:
   NOV 30, 2025
   NOV 15, 2025
   OCT 31, 2025
   ...

3. For each date you need (focus on 2024-2025 first):
   - Click the date link
   - Browser opens HTML table
   - Right-click → Save As → Save as HTML
   - OR copy table → paste into Excel
   - Name file: NSDL_FORTNIGHT_YYYYMMDD.html or .xlsx

4. Prioritize recent dates (2024-2025) = 48 files
   Then backfill 2023 if time permits = 24 files

5. Use the CSV file created above as checklist

ESTIMATED TIME:
- 2024-2025 fortnightly (48 files): 30 minutes
- 2023 fortnightly (24 files): 15 minutes
- Monthly aggregated (3 tables): 10 minutes
TOTAL: 55 minutes
""")

print("="*80)
print("FILES CREATED:")
print("  - nsdl_fortnightly_dates_to_download.csv (checklist)")
print("="*80)

NSDL FPI DATA DOWNLOADER (2023-2025)

[1] Downloading NSDL Monthly Data (2023-2025)...

MANUAL DOWNLOAD REQUIRED FOR MONTHLY DATA:
  1. Visit: https://www.fpi.nsdl.co.in/Reports/Yearwise.aspx?RptType=6
  2. Select year: 2023, then copy table to Excel
  3. Save as: nsdl_monthly_2023.xlsx
  4. Repeat for 2024 and 2025
  5. Takes 10 minutes total

[2] Generating fortnightly dates list...

Fortnightly dates to download:

2023:
  Jan 15, 2023 | Jan 31, 2023
  Feb 15, 2023 | Feb 28, 2023
  Mar 15, 2023 | Mar 31, 2023
  Apr 15, 2023 | Apr 30, 2023
  May 15, 2023 | May 31, 2023
  Jun 15, 2023 | Jun 30, 2023
  Jul 15, 2023 | Jul 31, 2023
  Aug 15, 2023 | Aug 31, 2023
  Sep 15, 2023 | Sep 30, 2023
  Oct 15, 2023 | Oct 31, 2023
  Nov 15, 2023 | Nov 30, 2023
  Dec 15, 2023 | Dec 31, 2023

Total 2023 dates: 24

2024:
Total 2024 dates: 24

2025 (Jan-Nov):
Total 2025 dates: 22

✓ Total fortnightly dates: 70
✓ Saved date list to: nsdl_fortnightly_dates_to_download.csv

DOWNLOAD INSTRUCTIONS

FORTNIGHT